In [ ]:
!pip install sentence_transformers
!pip install underthesea
from IPython.display import clear_output 
clear_output()
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import numpy as np
import math
import re
from sklearn.cluster import KMeans
from itertools import islice
from scipy.spatial import distance
import random
from underthesea import pos_tag
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def extract_importance_sentence(content, max_length, tokenizer):
    model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
    sentences = content.split(".")
    for i in range (len(sentences)):
        sentences[i] = sentences[i].strip()
    embeddings = model.encode(sentences)
    fixed_params = {"max_iter":300 }
    param_grid = {"n_clusters": 1, "random_state": 0, "algorithm": 'auto'}
    kmeans = KMeans(**param_grid)
    kmeans.fit(embeddings)
    center = kmeans.cluster_centers_[0]

    selected_embeddings = {}

    indexs = list(range(len(embeddings)))
    for j in range(len(sentences)):
        sum_distances = {}
        if j == 0:
            for i in indexs:
                distances = []

                distances.append(distance.euclidean(np.array(embeddings[i]), center))

                sum_distances[i] = distances
            sorted_sum_distances = dict(sorted(sum_distances.items(), key = lambda item: item[1]))
            idx = list(sorted_sum_distances.keys())[0]
            selected_embeddings[idx]= embeddings[idx]
            center_of_selection = selected_embeddings[idx]
            indexs.remove(idx)
        else:
            for i in indexs:
                distances = []

                distances.append(distance.euclidean(np.array(embeddings[i]), center))
                
                distances.append(distance.euclidean(np.array(embeddings[i]), center_of_selection))
                #print(distances)
                sum_distances[i] = np.mean(distances)
                #print(sum_distances)

            sorted_sum_distances = dict(sorted(sum_distances.items(), key = lambda item: item[1]))

            idx = list(sorted_sum_distances.keys())[0]
            selected_embeddings[idx]= embeddings[idx]
            indexs.remove(idx)
            kmeans = KMeans(**param_grid)
            kmeans.fit(np.array(list(selected_embeddings.values())))
            center_of_selection = kmeans.cluster_centers_[0]

    indexs = list(selected_embeddings.keys())   
    just_copy = indexs.copy()
    #print(indexs)   

    for index in just_copy:
        #print(len(re.split(r'[, -:]', sentences[index])))
        if count_token(sentences[index], tokenizer) > max_length:
            #print("remove", index)
            indexs.remove(index)
            #print(indexs)

  
    true_select = []
    length_sentences_chosen = 0
    for index in indexs:
        length_sentences_chosen += count_token(sentences[index], tokenizer)
        if length_sentences_chosen < max_length :
            true_select.append(index)
            #print(true_select)
        else: break

    selected_list_sentences = []

    true_select.sort()

    for index in true_select:
        selected_list_sentences.append(sentences[index])

    important_document = '. '.join(selected_list_sentences) + '.'
    return important_document

def count_token(sentences, tokenizer):
    token = tokenizer(sentences)
    return len(token["input_ids"])

In [ ]:
document =   "Đây là một trong những nội dung tại văn bản vừa được UBND TP Hà Nội ban hành về việc tăng cường công tác quản lý nuôi , giết mổ , kinh doanh và sử dụng thịt chó , mèo trên địa bàn .Theo đó , các sở , ngành trên địa bàn phải vào cuộc ngay để hướng tới thay đổi thói quen của người dân khi dùng chó , mèo làm thực phẩm .Gây phản cảm với du khách , người nước ngoàiCũng trong văn bản này , UBND TP Hà Nội thừa nhận rằng việc kinh doanh , giết mổ và sử dụng thịt chó , mèo tại Hà Nội thời gian qua đã tạo ra những hình ảnh phản cảm đối với khách tham quan du lịch , người nước ngoài đang sinh sống và làm việc tại Hà Nội , gây ảnh hưởng tới hình ảnh của một thủ đô "" văn minh , hiện đại "" .Trong thực tế , theo ghi nhận của Tuổi Trẻ tại phố Tam Trinh , ngay đoạn đầu cầu Mai Động ( quận Hoàng Mai ) , một đoạn phố dài với gần chục cửa hàng buôn bán thịt chó sống , nhà hàng phục vụ các món chế biến từ thịt chó vẫn hoạt động tấp nập nhiều năm nay .Chị Th . , một chủ sạp bán thịt chó sống ở đây , cho hay trong 10 ngày đầu mỗi tháng âm lịch , các cửa hàng vắng vẻ hơn vì thói quen "" kiêng "" ăn thịt chó vào đầu tháng tránh xui xẻo .Riêng những ngày cuối tháng , các cửa hàng ở đây tấp nập người mua bán .Theo chị Th . , trung bình mỗi tháng chị bán được khoảng 3 tạ thịt chó sống .Tương tự , tại phố Nguyễn Khang ( quận Cầu Giấy ) , hàng loạt nhà hàng thịt chó quy mô lớn ( 1-5 tầng ) như Sơn Hải , Chiếu Hoa , Mơ Hoa quán ... luôn tấp nập , mỗi ngày tiếp cả nghìn thực khách .Tuy nhiên , hàng chục quán thịt chó trên đường Âu Cơ ( phường Nhật Tân , quận Tây Hồ ) từng nổi tiếng một thời như Trần Mục , Hồ Kiếm , A Trang , Anh Tú Xịn , Anh Tú Nhà Kính , Anh Tú Nhà Lá ... đã đóng cửa không rõ lý do , chỉ còn duy nhất cửa hàng có tên Anh Tú Béo .Nhiều nguy cơ lây lan dịch bệnhTheo chuyên gia truyền thông Phan Kiền , chó không phải là loài động vật nằm trong danh mục cấm săn bắt , mua bán và giết mổ , thịt chó thương phẩm cũng không phải là mặt hàng thực phẩm cấm lưu hành nên rất khó dùng luật để cấm .Do đó , việc Hà Nội khuyến khích người dân hạn chế , tiến tới không ăn thịt chó là cách làm vừa văn minh vừađúng luật ."" Hoàn toàn có thể thay đổi một thói quen xấu nếu được truyền thông tốt , mà việc bỏ ăn tiết canh là một ví dụ .Trước đây các hàng quán "" lòng lợn , tiết canh "" mọc khắp nơi bởi đây là món khoái khẩu của dân nhậu .Nhưng khi báo chí đưa tin nhiều về các ca nhiễm khuẩn cầu lợn gây tử vong với những triệu chứng khủng khiếp , gần như món ăn này bị tẩy chay , hiện rất ít người dùng món ăn này "" - vị này nói .Trong khi đó , không chỉ những bạn trẻ nuôi chó cảnh làm thú cưng phản đối việc xem chó là một loại thực phẩm , mà ngay cả những người từng xem thịt chó là món "" khoái khẩu "" cũng ủng hộ việc bỏ ăn thịt chó .Dù từng ăn thịt chó , anh Nguyễn Văn Thuận , một nhân viên văn phòng ở quận Thanh Xuân , cũng ủng hộ chủ trương của Hà Nội .Theo anh Thuận , một khi thịt chó còn thịnh hành và có giá trị thương phẩm cao , nạn trộm chó vẫn tồn tại và khó tránh khỏi cảnh người trộm chó bị đánh "" thừa sống thiếu chết "" như đã từng xảy ra .Ngoài ra nguy cơ lây lan dịch bệnh , đặc biệt là bệnh dại , do hoạt động buôn bán và vận chuyển chó sống giữa các địa phương cũng là vấn đề đáng lo ngại .Do đó , muốn hạn chế nạn trộm chó và ngăn ngừa nguy cơ lây lan dịch bệnh , cần khuyến khích không nên ăn thịt chó ."" Thịt chó cũng ngon và lạ miệng nhưng nếu cấm ăn thịt chó , mình thấy cũng không vấn đề gì vì còn rất nhiều loại thực phẩm khác thay thế "" - anh Thuận nói ."
sum = "Các quận , huyện , thị xã tuyên truyền bằng nhiều hình thức để người dân từ bỏ thói quen ăn thịt chó , mèo nhằm tránh nguy cơ mắc bệnh truyền nhiễm ( bệnh dại , xoắn khuẩn , tả ... ) cũng như không gây ảnh hưởng tới hình ảnh thủ đô văn minh , hiện đại ."
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word",use_fast = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
short = extract_importance_sentence(document, 100, tokenizer)

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/sumarization/bio_medicine.csv')

In [ ]:
plt.figure(figsize=(12, 6))
plt.style.use('ggplot')
plt.subplot(1, 2, 1)
sns.distplot(df['Document'].apply(lambda x: count_token(x, tokenizer)))
plt.title('Distribution of Document token length')
plt.xlabel('Number of tokens')

In [ ]:

def add_prompt(text):
    words = pos_tag(text)
    txt = ""
    for word in words:
        if word[1] in ["N", "V", "A"]:
            txt += word[0] + " {} ".format(word[1])
       
        else:
            txt += word[0] + " "
    return txt

In [ ]:
#shortest document
min = 1000
for idx, row in tqdm(enumerate(df.itertuples())):
    doc = row[1]
    cur = count_token(doc, tokenizer)
    if  cur < min:
        min = cur
        print(idx)
print(min)
   

40it [00:00, 203.82it/s]

0
1
3
20


321it [00:01, 274.06it/s]

283
314


3642it [00:12, 263.45it/s]

...

5935
5956


6113it [00:21, 233.74it/s]

6085


10652it [00:42, 251.02it/s]

21


In [ ]:
df["Document"][5935]

'Nếu bạn nhìn kỹ, bạn sẽ nhận thấy rằng miệng của nó đang bám trên da của bạn, và cơ thể của nó nằm ở phía sau. Bạn cũng có thể bôi cồn tẩy rửa, oxy già, hoặc i-ốt vào da. Rửa kỹ vùng da bị bọ ve cắn cũng như bàn tay của bạn. Một vài con bọ khá nhỏ đến nỗi bạn khó có thể loại bỏ chúng thông qua các phương pháp thông thường. Bác sĩ sẽ có thể giúp bạn loại bỏ bọ ve khỏi da.'

In [ ]:
df["Summary"][5935]

'Xác định phần đầu của bọ ve. Rửa sạch vết cắn với nước và xà phòng. Đi khám nếu bạn gặp khó khăn trong việc loại bỏ bọ ve khỏi da.'

In [ ]:
count_token('Bạn cũng có thể bôi cồn tẩy rửa, oxy già, hoặc i-ốt vào da. Rửa kỹ vùng da bị bọ ve cắn cũng như bàn tay của bạn. Bác sĩ sẽ có thể giúp bạn loại bỏ bọ ve khỏi da.', tokenizer)

49

In [ ]:
count_token(df["Document"][5935])

103

In [ ]:
extract_importance_sentence(df["Document"][5935], 70, tokenizer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Bạn cũng có thể bôi cồn tẩy rửa, oxy già, hoặc i-ốt vào da. Rửa kỹ vùng da bị bọ ve cắn cũng như bàn tay của bạn. Bác sĩ sẽ có thể giúp bạn loại bỏ bọ ve khỏi da.'

In [ ]:
#sample: add prompt to text
add_prompt('Bạn cũng có thể bôi cồn tẩy rửa, oxy già, hoặc i-ốt vào da. Rửa kỹ vùng da bị bọ ve cắn cũng như bàn tay của bạn. Bác sĩ sẽ có thể giúp bạn loại bỏ bọ ve khỏi da.')

In [ ]:
#check
num = 0
for idx, row in tqdm(enumerate(df.itertuples())):
    doc = row[1]
    if count_token(doc, tokenizer) > 512:
        num += 1
print(num)
   

In [ ]:
#apply add_prompt for all document < 512 token length (using herding to make text short)
df["Document"] = df["Document"].apply(lambda x: add_prompt(x) if count_token(x, tokenizer) < 512 else add_prompt(extract_importance_sentence(x, 512, tokenizer)) )

In [4]:
from tqdm import tqdm

In [ ]:
df.to_csv("/kaggle/working/new")

In [ ]:
df["Document"][0]